In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

D:\anaconda\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\pyth

In [2]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
#运行次数
max_steps = 150
#图片数量
image_num =3000
#文件路径
DIR="F:/yjs2019/Tensorflow_learn/"
#定义会话
sess=tf.Session()
#载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable = False , name='embedding')
# #每个批次的大小
# batch_size = 100
#计算一共有多少个批次
# n_batch = mnist.train.num_examples // batch_size

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram',var)#直方图
        
#命名空间
with tf.name_scope('input'):
    #定义两个placeholder
    x = tf.placeholder(tf.float32,[None,784],name = 'x-input')
    y = tf.placeholder(tf.float32,[None,10],name = 'y-input')
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.Variable(0.001, dtype = tf.float32)
#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10)
with tf.name_scope('layer'):
    with tf.name_scope('layer1'):
        #创建一个简单的神经网络
        with tf.name_scope('wights1'):
            W1 = tf.Variable(tf.truncated_normal([784,500],stddev=0.1),name = 'W1')
            variable_summaries(W1)
        with tf.name_scope('biases'):
            b1 = tf.Variable(tf.zeros([500])+0.1,name='b1')
            variable_summaries(b1)
        with tf.name_scope('tanh1'):
            L1 = tf.nn.tanh(tf.matmul(x,W1)+b1)
        with tf.name_scope('drop1'):
            L1_drop = tf.nn.dropout(L1,keep_prob )
        with tf.name_scope('wights2'):
            W2 = tf.Variable(tf.truncated_normal([500,300],stddev=0.1),name='W2')
            variable_summaries(W2)
        with tf.name_scope('biases'):
            b2 = tf.Variable(tf.zeros([300])+0.1,name='b2')
            variable_summaries(b2)
        with tf.name_scope('tanh2'):
            L2 = tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
        with tf.name_scope('drop2'):
            L2_drop = tf.nn.dropout(L2,keep_prob )
        with tf.name_scope('wights3'):
            W3 = tf.Variable(tf.truncated_normal([300,10],stddev=0.1),name='W3')
            variable_summaries(W2)
        with tf.name_scope('biases3'):
            b3 = tf.Variable(tf.zeros([10])+0.1,name='b3')
            variable_summaries(b3)
        with tf.name_scope('tanh'):
            prediction = tf.nn.tanh(tf.matmul(L2_drop,W3)+b3)

#二次代价函数
with tf.name_scope('loss'):
# loss = tf.reduce_mean(tf.square(y-prediction))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y , logits = prediction))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    #使用梯度下降法
#     train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    train_step = tf.train.AdamOptimizer(lr).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()
sess.run(init)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值
        #求准确率
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

#产生metadata文件
if tf.gfile.Exists(DIR+'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR+'projector/projector/metadata.tsv')
with open(DIR+'projector/projector/metadata.tsv','w') as f:
    labels=sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i])+'\n')
#合并所有的summary
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR+'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR+'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)

for epoch in range(max_steps):
    #每个批次100个样本
    sess.run(tf.assign(lr,0.003 * (0.95 ** epoch)))
    batch_xs,batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
#     sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
#     summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    summary,_=sess.run([merged,train_step],feed_dict = {x:batch_xs,y:batch_ys,keep_prob:1.0},options = run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%3d'%epoch)
    projector_writer.add_summary(summary, epoch)
    learning_rate = sess.run(lr)
    test_acc = sess.run(accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
    train_acc = sess.run(accuracy,feed_dict = {x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
    if epoch%1 == 0:
        print("Iter" + str(epoch)+",Testing Accuaracy"+str(test_acc)+",Training Accuracy"+str(train_acc)+",Learning Rate" +str(learning_rate))
saver.save(sess,DIR+'projector/projector/a_model.ckpt',global_step = max_steps)
projector_writer.close()
sess.close()

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter0,Testing Accuaracy0.29,Training Accuracy0.2797091,Learning Rate0.003
Iter1,Testing Accuaracy0.4726,Training Accuracy0.46254545,Learning Rate0.00285
Iter2,Testing Accuaracy0.5456,Training Accuracy0.5410182,Learning Rate0.0027075
Iter3,Testing Accuaracy0.6006,Training Accuracy0.5958,Learning Rate0.002572125
Iter4,Testing Accuaracy0.6242,Training Accuracy0.6248364,Learning Rate0.0024435187
Iter5,Testing Accuaracy0.6477,Training Accuracy0.64585453,Learning Rate0.0023213427
Iter6,Testing Accuaracy0.655,Training Accuracy0.6513636,Learning Rate0.0022052757
Iter7,Testing Accuaracy0.6671,Training Accuracy0.6635818,Learning Rate0.002095012
Iter8,Testing Accuaracy0.6846,Training Accuracy0.68316364,Learning Rate0.0019902613
Iter9,Testing Accuaracy0.7332,Training Accuracy0.73236364,Learning Rate0.001890

Iter95,Testing Accuaracy0.8942,Training Accuracy0.88618183,Learning Rate2.2954284e-05
Iter96,Testing Accuaracy0.8942,Training Accuracy0.8863091,Learning Rate2.180657e-05
Iter97,Testing Accuaracy0.8942,Training Accuracy0.8863636,Learning Rate2.0716241e-05
Iter98,Testing Accuaracy0.8941,Training Accuracy0.8863636,Learning Rate1.968043e-05
Iter99,Testing Accuaracy0.8944,Training Accuracy0.8864727,Learning Rate1.8696408e-05
Iter100,Testing Accuaracy0.8944,Training Accuracy0.8865091,Learning Rate1.7761588e-05
Iter101,Testing Accuaracy0.8945,Training Accuracy0.8865091,Learning Rate1.6873508e-05
Iter102,Testing Accuaracy0.8945,Training Accuracy0.8864727,Learning Rate1.6029833e-05
Iter103,Testing Accuaracy0.8944,Training Accuracy0.8865091,Learning Rate1.5228341e-05
Iter104,Testing Accuaracy0.8944,Training Accuracy0.88656366,Learning Rate1.4466924e-05
Iter105,Testing Accuaracy0.8945,Training Accuracy0.8865091,Learning Rate1.3743578e-05
Iter106,Testing Accuaracy0.8945,Training Accuracy0.88658184